In [1]:
import numpy as np
import os

In [8]:
### CLASSES ###
class Board:
    def __init__(self, file_name):
        # Get Boards folder
        current_directory = os.getcwd() #Current working directory
        parent_directory = os.path.dirname(current_directory)
        new_folder_path = os.path.join(parent_directory, 'Unsolved-Boards')
        text_file_path = os.path.join(new_folder_path, file_name)

        # Check if the folder exists, if not, create it
        if not os.path.exists(text_file_path):
            print("ERROR - the path " + text_file_path + " does not exist")
            return

        
        # Read File
        fileReader = open(text_file_path, "r")    
        file_as_text =fileReader.read()
        text_to_array = file_as_text.split('\n')
        text_to_array = np.array([[tile for tile in row] for row in text_to_array if len(row) > 0])
        #print(text_to_array)
        
        
        # Turn File into Board object
        self.board = []
        
        for row in text_to_array:
            row_list = []
            for element in row:
                row_list.append(Cell(element))
                
            self.board.append(row_list)
        
    def print_board(self):
        for row in self.board:
            string = ""
            
            for cell in row:
                string += str(cell.value) + " "
                
            print(string)

        
class Cell:
    def __init__(self, input_val):
        self.value = 0
        if(input_val != '*'):
            self.value = input_val

In [9]:
game = Board("Puzzle_1.txt")
game.print_board()

8 2 1 0 0 0 0 0 7 
0 0 0 8 0 0 0 6 0 
0 6 0 9 3 0 0 0 5 
0 0 8 2 0 1 6 0 0 
0 0 0 7 0 0 2 8 4 
2 4 0 6 0 3 7 0 0 
6 0 5 0 0 0 1 0 3 
0 7 0 0 5 0 0 0 0 
9 1 2 0 0 0 0 0 6 


In [ ]:
### BACK TRACKING ALGORTHM ###

In [ ]:
### SOLVING STRATEGIES ###

In [ ]:
### FRONT END CONNECTION ###

In [ ]:
### COMBINED SOLVING AND MAIN FUNCTION ###